# Библиотеки

In [29]:
from zlib import crc32

import numpy as np
import pandas as pd
import scipy.stats as st
import re

from nltk import lm
from nltk.util import ngrams as nltk_ngrams
from nltk.tag.hmm import HiddenMarkovModelTrainer

# Код генерации выборки

In [2]:
EMAIL_REGEX = re.compile(r"[^@]+@phystech.edu")

def generate_dataset(code):
    rs = np.random.RandomState(code)
    length = rs.randint(low = 25, high = 75)
    proba = rs.uniform()
    x = [(rs.uniform() > 0.5) * 1.0]
    for i in range(length):
        if rs.uniform() < proba:
            x.append(x[-1])
        else:
            x.append(1 - x[-1])
    return np.array(x)

# Генерация выборки для вашей почты

<span style="color:red">
    ВАЖНО!
    Почта, которую укажете ниже и почта с которой Вы отправляете домашнее задание должна совпадать!
    В момент проверки задания алгоритм генерирует выборку на основе почты из анкеты!
</span>

Внимательно проверьте почту для которой выполняется задание!

In [3]:
task = dict()
task['mail'] = input(prompt='Enter your mail: ')
assert EMAIL_REGEX.match(task['mail']), 'Not a mail in the phystech.edu domain'
task['id'] = crc32(task['mail'].encode('utf-8'))
task['data'] = generate_dataset(task['id'])

task

Enter your mail: bogdanov.ai@phystech.edu


{'mail': 'bogdanov.ai@phystech.edu',
 'id': 1162440694,
 'data': array([1., 0., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1.,
        0., 0., 0., 1., 0., 1., 1., 0., 0., 1., 1., 1., 0., 0., 1., 1., 1.,
        0.])}

# Работа с выборкой

In [72]:
data = task['data']

## Оценка p(X_{i+1}= 0|X_{i}=0) методом максимального правдоподобия

In [102]:
tokens = list(map(str, data))
vocab = sorted(map(str, list(set(data))))

In [103]:
def make_tag(c):
    return (c, c)

In [104]:
supervised = [make_tag(c) for c in data]
trainer = HiddenMarkovModelTrainer(range(2), vocab)
tagger = trainer.train_supervised([supervised])

In [105]:
print(f'Оценка p(0|0) = {tagger._transitions[0].prob(0):.3f}')

Оценка p(0|0) = 0.611


## Отвергается ли нулевая гипотеза p(X_{i+1}= 0|X_{i}=0)=0.9?

In [109]:
T = 0
for i in range(len(data) - 1):
    if data[i] == 0 and data[i] == data[i+1]:
        T += 1

In [114]:
n = len(data)
p0 = 0.9
st.binomtest(T, n, p0, alternative='less').pvalue

1.3761493574495718e-16

Отвергается

## Какова вероятность встретить Xi=2 с применением сглаживания Лапласа?

In [107]:
laplace = lm.Laplace(order=1)
laplace.fit([list(nltk_ngrams(tokens, 1))], vocabulary_text=vocab)

In [108]:
print(f"Вероятность встретить X_i = 2 с Лапласом: {laplace.score('2'):.2f}")

Вероятность встретить X_i = 2 с Лапласом: 0.03
